In [1]:
%pip install streamlit langchain faiss-cpu pypdf google-generativeai python-dotenv

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/10.2 MB 5.8 MB/s eta 0:00:02
   --------- ------------------------------ 2.4/10.2 MB 4.7 MB/s eta 0:00:02
   ----------- ---------------------------- 2.9/10.2 MB 3.9 MB/s eta 0:00:02
   --------------- ------------------------ 3.9/10.2 MB 4.3 MB/s eta 0:00:02
   ------------------ --------------------- 4.7/10.2 MB 4.2 MB/s eta 0:00:02
   --------------------- ------------------ 5.5/10.2 MB 4.0 MB/s eta 0:00:02
   ------------------------ --------------- 6.3/10.2 MB 4.0 MB/s eta 0:00:01
   --------------------------- ------------ 7.1/10.2 MB 3.9 MB/s eta 0:00:01
   -----

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 3.0.0 requires google-ai-generativelanguage<1.0.0,>=0.7.0, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [ ]:
# gemini_rag_filepicker.py

import os
import uuid
from datetime import datetime
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.language_models import LLM
import tkinter as tk
from tkinter import filedialog

# ✅ Load environment variables
load_dotenv()

# 🔐 API Keys
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Initialize Gemini model and embeddings
llm = GoogleGenerativeAI(
    model="models/gemini-2.0-flash",
    google_api_key=GEMINI_API_KEY,
    temperature=0.3
)

embedding_model = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=GEMINI_API_KEY
)

# Prepare output directory
os.makedirs("output_logs", exist_ok=True)

def open_file_picker(multiple=False):
    root = tk.Tk()
    root.withdraw()  # Hide root window
    root.attributes("-topmost", True)  # Bring dialog to front

    if multiple:
        files = filedialog.askopenfilenames(filetypes=[("PDF Files", "*.pdf")], parent=root)
    else:
        file = filedialog.askopenfilename(filetypes=[("PDF Files", "*.pdf")], parent=root)
        files = [file] if file else []
    
    root.destroy()  # Clean up the hidden root window
    return list(files)

def load_documents(file_paths):
    docs = []
    filenames = []
    for file in file_paths:
        loader = PyPDFLoader(file)
        docs.extend(loader.load())
        filenames.append(os.path.basename(file))
    return docs, filenames

def save_output(question, answer, filenames):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_id = uuid.uuid4().hex[:6]
    out_path = f"output_logs/output_{file_id}_{timestamp}.txt"
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(f"Files: {filenames}\n")
        f.write(f"Question: {question}\n")
        f.write(f"Answer: {answer}\n")
    return out_path

def main():
    print("📄 Gemini RAG Console App with File Picker")
    print("1. Upload single file")
    print("2. Upload multiple files")
    choice = input("Choose option (1 or 2): ").strip()

    if choice == "1":
        file_paths = open_file_picker(multiple=False)
    elif choice == "2":
        file_paths = open_file_picker(multiple=True)
    else:
        print("Invalid option.")
        return

    if not file_paths or file_paths[0] == "":
        print("❌ No files selected.")
        return

    docs, filenames = load_documents(file_paths)
    print(f"✅ Loaded {len(filenames)} file(s), {len(docs)} document chunks.")

    vectordb = FAISS.from_documents(docs, embedding_model)
    retriever = vectordb.as_retriever()
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

    question = input("\n💬 Ask your question: ").strip()
    print("🔍 Generating answer...")
    answer = qa_chain.run(question)

    print("\n📘 Gemini's Answer:\n")
    print(answer)

    out_path = save_output(question, answer, ", ".join(filenames))
    print(f"\n✅ Answer saved to: {out_path}")

if __name__ == "__main__":
    main()